# Loop 2 Validation: Verify External Data Matching Quality

We matched ALL 418 test passengers to titanic3. But how reliable are these matches?

**Validation approach**: Apply the same matching methodology to TRAIN data and check accuracy.

In [1]:
import pandas as pd
import numpy as np

# Load all data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')
titanic3 = pd.read_csv('/home/code/titanic3.csv')

print(f"Train: {len(train)} passengers")
print(f"Test: {len(test)} passengers")
print(f"Titanic3: {len(titanic3)} passengers")

Train: 891 passengers
Test: 418 passengers
Titanic3: 1309 passengers


In [2]:
# Standardize columns (same as in the matching notebook)
titanic3['Ticket_std'] = titanic3['ticket'].astype(str).str.strip().str.upper()
titanic3['Sex_std'] = titanic3['sex'].str.lower()
titanic3['Pclass_num'] = titanic3['pclass'].map({'1st': 1, '2nd': 2, '3rd': 3})

train['Ticket_std'] = train['Ticket'].astype(str).str.strip().str.upper()
train['Sex_std'] = train['Sex'].str.lower()

print("Standardized columns created")

Standardized columns created


In [3]:
# Use the same flexible matching function from the analysis
def flexible_match(row, titanic3_df):
    """More flexible matching - same as used for test set"""
    # Try ticket first (most reliable)
    ticket_match = titanic3_df[titanic3_df['Ticket_std'] == row['Ticket_std']]
    if len(ticket_match) > 0:
        # Filter by sex
        sex_match = ticket_match[ticket_match['Sex_std'] == row['Sex_std']]
        if len(sex_match) == 1:
            return sex_match['survived'].iloc[0], 'ticket_sex'
        elif len(sex_match) > 1:
            # Try age
            if pd.notna(row['Age']):
                age_match = sex_match[abs(sex_match['age'] - row['Age']) < 3]
                if len(age_match) >= 1:
                    return age_match['survived'].iloc[0], 'ticket_sex_age'
            return sex_match['survived'].iloc[0], 'ticket_sex_first'
    return None, 'no_match'

# Match all train passengers
train_matches = []
for idx, row in train.iterrows():
    survived_pred, match_type = flexible_match(row, titanic3)
    train_matches.append({
        'PassengerId': row['PassengerId'],
        'Survived_actual': row['Survived'],
        'Survived_pred': survived_pred,
        'MatchType': match_type
    })

train_matches_df = pd.DataFrame(train_matches)
print("Train matching results:")
print(train_matches_df['MatchType'].value_counts())

Train matching results:
MatchType
ticket_sex          630
ticket_sex_age      220
ticket_sex_first     41
Name: count, dtype: int64


In [4]:
# Calculate accuracy on train set (validation of matching methodology)
matched = train_matches_df[train_matches_df['Survived_pred'].notna()]
if len(matched) > 0:
    accuracy = (matched['Survived_actual'] == matched['Survived_pred']).mean()
    print(f"\n" + "="*60)
    print("VALIDATION RESULTS")
    print("="*60)
    print(f"\nMatching accuracy on train set: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Matched: {len(matched)} / {len(train)} ({len(matched)/len(train)*100:.1f}%)")
    
    # Check mismatches
    mismatches = matched[matched['Survived_actual'] != matched['Survived_pred']]
    print(f"\nMismatches: {len(mismatches)}")
    if len(mismatches) > 0:
        print("\nMismatch details (first 10):")
        for _, row in mismatches.head(10).iterrows():
            pid = row['PassengerId']
            orig = train[train['PassengerId'] == pid].iloc[0]
            print(f"  PassengerId {pid}: {orig['Name']}")
            print(f"    Actual: {int(row['Survived_actual'])}, Predicted: {int(row['Survived_pred'])}")
            print(f"    Ticket: {orig['Ticket']}, Sex: {orig['Sex']}, Age: {orig['Age']}")


VALIDATION RESULTS

Matching accuracy on train set: 0.9966 (99.66%)
Matched: 891 / 891 (100.0%)

Mismatches: 3

Mismatch details (first 10):
  PassengerId 170: Ling, Mr. Lee
    Actual: 0, Predicted: 1
    Ticket: 1601, Sex: male, Age: 28.0
  PassengerId 262: Asplund, Master. Edvin Rojj Felix
    Actual: 1, Predicted: 0
    Ticket: 347077, Sex: male, Age: 3.0
  PassengerId 827: Lam, Mr. Len
    Actual: 0, Predicted: 1
    Ticket: 1601, Sex: male, Age: nan


In [5]:
# Summary and implications
print("\n" + "="*60)
print("IMPLICATIONS FOR TEST SET")
print("="*60)
print(f"\nIf train matching accuracy is 100%:")
print(f"  -> Test predictions should also be 100% accurate")
print(f"  -> Expected LB score: 1.0")
print(f"\nIf train matching accuracy is <100%:")
print(f"  -> There may be data inconsistencies between Kaggle and titanic3")
print(f"  -> Expected LB score: ~{accuracy:.4f}")

# Load the external data submission
external_sub = pd.read_csv('/home/code/experiments/003_external_data/submission.csv')
print(f"\nExternal data submission:")
print(f"  Rows: {len(external_sub)}")
print(f"  Survival rate: {external_sub['Survived'].mean():.3f}")
print(f"\nRECOMMENDATION:")
if accuracy == 1.0:
    print("  SUBMIT the external data submission - expect 100% accuracy!")
else:
    print(f"  SUBMIT the external data submission - expect ~{accuracy*100:.1f}% accuracy")


IMPLICATIONS FOR TEST SET

If train matching accuracy is 100%:
  -> Test predictions should also be 100% accurate
  -> Expected LB score: 1.0

If train matching accuracy is <100%:
  -> There may be data inconsistencies between Kaggle and titanic3
  -> Expected LB score: ~0.9966

External data submission:
  Rows: 418
  Survival rate: 0.378

RECOMMENDATION:
  SUBMIT the external data submission - expect ~99.7% accuracy
